# Build a simple Chatbot with stored memory using LangChain

* A chatbot app is a software application designed to simulate conversation with human users. It uses AI to understand what someone is saying and to respond with application answers. These apps can be used for various purposes like customer support, providing information, or entertainment. Essentially, it's like texting with a program that can answer questions and help with tasks.

##### In this project, we will use the LangChain legacy chain LLMChain. It works well, but LangChain displays a nasty deprecation warning. To avoid it, we will enter the following code:

In [1]:
import warnings
from langchain._api import LangChainDeprecationWarning
warnings.simplefilter("ignore", category=LangChainDeprecationWarning)

In [2]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
groq_api_key = os.environ["GROQ_API_KEY"]

* For this project, we will use OpenSource llama3-70b-8192 model.

In [3]:
from langchain_groq import ChatGroq
chatbot = ChatGroq(model="llama3-70b-8192")

* Human Message: The user input.

In [4]:
from langchain_core.messages import HumanMessage

messagetotheChatbot = [
    HumanMessage(content="My favorite color is Green.")
]

##### Call the LLM model.

In [5]:
chatbot.invoke(messagetotheChatbot)

AIMessage(content="Green is a wonderful color! It's a calming and natural hue that can evoke feelings of growth, harmony, and balance. Did you know that green is also associated with emotions, such as freshness, hope, and renewal?\n\nWhat is it about the color green that you love the most? Is it the bright, lime green, the deep, forest green, or maybe the soft, minty green?\n\nAlso, do you have a favorite way of incorporating green into your daily life, such as wearing green clothes, decorating with green accents, or enjoying nature and the outdoors?", response_metadata={'token_usage': {'completion_tokens': 116, 'prompt_tokens': 16, 'total_tokens': 132, 'completion_time': 0.343677903, 'prompt_time': 0.003598524, 'queue_time': 0.015006884, 'total_time': 0.347276427}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_2f30b0b571', 'finish_reason': 'stop', 'logprobs': None}, id='run-3e30dbd5-571f-4634-91f3-9663e66042d9-0', usage_metadata={'input_tokens': 16, 'output_tokens': 116, 

##### Check if the ChatBot remembers your favorite color

In [6]:
chatbot.invoke([
    HumanMessage(content="What is my favorite color?")
])

AIMessage(content="I'm just a language model, I don't have any information about you or your personal preferences, so I can't tell you what your favorite color is. However, I can ask you what your favorite color is if you'd like to share!", response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 16, 'total_tokens': 67, 'completion_time': 0.145714286, 'prompt_time': 0.003309239, 'queue_time': 0.015810469, 'total_time': 0.149023525}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_753a4aecf6', 'finish_reason': 'stop', 'logprobs': None}, id='run-b06a43c5-04c7-4ee5-bc17-d32ed5f95cc7-0', usage_metadata={'input_tokens': 16, 'output_tokens': 51, 'total_tokens': 67})

* As you can see, our ChatBot cannot remember our previous interaction.

#### Add Memory to the ChatBot.

In [7]:
from langchain import LLMChain
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.prompts import HumanMessagePromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.memory import FileChatMessageHistory

In [8]:
memory = ConversationBufferMemory(
    chat_memory= FileChatMessageHistory("messages.json"),
    memory_key="messages",
    return_messages=True
)

In [9]:
prompt = ChatPromptTemplate(
    input_variable = ["content", "messages"],
    messages=[
        MessagesPlaceholder(variable_name="messages"),
        HumanMessagePromptTemplate.from_template("{content}")
    ]
)

In [10]:
chain = LLMChain(
    llm=chatbot,
    prompt=prompt,
    memory=memory
)

In [11]:
chain.invoke("hello!")

{'content': 'hello!',
 'messages': [],
 'text': "Hello! It's nice to meet you. Is there something I can help you with, or would you like to chat?"}

In [12]:
chain.invoke("My name is Lokesh")

{'content': 'My name is Lokesh',
 'messages': [HumanMessage(content='hello!'),
  AIMessage(content="Hello! It's nice to meet you. Is there something I can help you with, or would you like to chat?")],
 'text': "Nice to meet you, Lokesh! It's great to have you here. How's your day going so far?"}

In [13]:
chain.invoke("What is my name?")

{'content': 'What is my name?',
 'messages': [HumanMessage(content='hello!'),
  AIMessage(content="Hello! It's nice to meet you. Is there something I can help you with, or would you like to chat?"),
  HumanMessage(content='My name is Lokesh'),
  AIMessage(content="Nice to meet you, Lokesh! It's great to have you here. How's your day going so far?")],
 'text': 'I remember! Your name is Lokesh.'}